In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from data_utils import get_data
from vae import VAE
import os
import numpy as np
import torch

In [ ]:
# Extraire les images et les labels
images, labels = [], []

events = [#'cards', 
          #'center'
          #'free-kick'
          #'cards',
          'To Subtitue'
         ]

train_images, train_labels = get_data(folder='train', events=events)

In [ ]:
import torch.optim as optim
import torch.utils.data as data
from vae import recon_loss, vae_loss

train_dataset = train_images
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialisation du modèle, de l'optimiseur et des paramètres
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae_instance = VAE().to(device)
optimizer = optim.Adam(vae_instance.parameters(), lr=1e-3)

num_epochs = 30
vae_instance.train()
for epoch in range(num_epochs):
    total_loss = 0
    for imgs in train_loader:
        imgs = imgs.to(device)
        optimizer.zero_grad()
        recon_imgs, mu, logvar = vae_instance(imgs)
        loss = vae_loss(recon_imgs, imgs, mu, logvar)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_dataset):.4f}")

# entrainer sur 100 epoques

In [ ]:
# threshold VAE loss: the value of 328 as the threshold for the loss gives the best distinction between categories

# the images of seven events(corner ok, penalty ok, free kick ok, red card, yellow card, tackle ok, substitute) defined from 
# the SEV dataset are selected and given to the VAE
# network as training data